## Modelo: **GPT-4o-mini**
---

In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [13]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

MODEL = "gpt-4o-mini"

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
embeddings = OpenAIEmbeddings()  #(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

#chain = model | parser 
#chain.invoke("Tell me a joke")

### Prompt template

In [15]:
from langchain.prompts import PromptTemplate

template = """
Eres un representatne de servicio al cliente, responde la pregunta basado en el contexto de abajo. 
Si no puedes responder la pregunta, responde "No lo se"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [16]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader("docs/manual-lpcr/manual-lpcr.md")
pages = loader.load_and_split()

In [17]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
    )

# Instantiate a retriever
retriever = vectorstore.as_retriever()

In [18]:
from langchain_core.runnables import RunnablePassthrough

md_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
}
| prompt
| model
| parser
)

In [19]:
questions = [
    "Como peso un producto?",
    "Como se introduce el precio por kilogramo?",
    "Como puedo borrar la venta total?",
]

for question in questions:
    print(question)
    print(md_chain.invoke(question))
    print()

Como peso un producto?
Para pesar un producto, sigue estos pasos:

1. Enciende la báscula presionando la tecla ON. Verás "------" en el visor/pantalla por unos segundos y luego mostrará ceros. En este momento, la báscula está lista para pesar.
2. Coloca el producto que deseas pesar en el plato de la báscula.
3. Observa el peso en el indicador de PESO. El indicador de CERO desaparecerá y mostrará el peso del objeto colocado.

¡Y eso es todo! Así puedes pesar un producto.

Como se introduce el precio por kilogramo?
El precio por kilogramo del producto se introduce presionando los números del teclado para indicar el valor deseado. Este valor aparecerá en el indicador de PRECIO y el valor a pagar se calculará automáticamente mostrándose en el indicador de TOTAL.

Como puedo borrar la venta total?
Para borrar la Venta Total (hacer 0 la Venta Total), primero debes desplegar la Venta Total y, una vez que se muestre el total, presionar la secuencia de teclas NC, PUNTO (.).

